<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project: Harmony
## 2.1 Processing Canva Palette
> Authors: Eugene Matthew Cheong
---

## Table of Contents ##

#### 1. Web Scraping

- [1.1 Scraping Lian Seng Hin Website](1.1_web_scraping_liansenghin.ipynb)
- [1.2 Scraping Hafary Website](1.2_web_scraping_hafary.ipynb)
- [1.3 Scraping Lamitak Website](1.3_web_scraping_lamitak.ipynb)
- [1.4 Scraping Nippon Website](1.4_web_scraping_nippon.ipynb)
- [1.5 Consolidate All Product Database](1.5_consolidate_product_database.ipynb)

#### 2. Preprocessing

- [2.1 Processing Canva Palettes](2.1_processing_canva_palette.ipynb)

#### 3. Modelling

- [3.1 Matching Input Photo to Products](3.1_matching_input_photo_to_products.ipynb)
- [3.2 Recommending Canva Palette to Products](3.2_recommending_canva_palette_to_product.ipynb)
- [3.3 Recommending Colours and Colour Palettes with Llama3](3.3_recommending_colours_and_colour_palettes_with_llama3.ipynb)

---

# Import Modules

In [1]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb
from skimage.color import rgb2lab, lab2rgb, rgb2hsv, hsv2rgb

from scipy.spatial.distance import euclidean, cosine, cityblock


# Generating PNG

Generating PNGs to be able to bring to the recommendation model

In [2]:
data_folder = "../datasets/"
data_img_folder = os.path.join(data_folder,"images")
canva_img_folder =  os.path.join(data_img_folder,"canva")
product_df = pd.read_csv(os.path.join(data_folder,"color_palette_df.csv"))

In [3]:
product_df

,Name,Color 1,Color 2,Color 3,Color 4
0,Rosettes and Cream,#EF7C8E,#FAE8E0,#B6E2D3,#D8A7B1
1,Rosy Flamingo,#E8B4B8,#EED6D3,#A49393,#67595E
2,Summer Splash,#05445E,#189AB4,#75E6DA,#D4F1F4
3,Pastel Dreams,#FBE7C6,#B4F8C8,#A0E7E5,#FFAEBC
4,Room for Comfort,#E7D2CC,#B9B7BD,#868B8E,#EEEDE7
...,...,...,...,...,...
515,Surprising Images,#EED6D3,#69C9BC,#F7BCA5,#F7837C
516,Wrap Around Style,#0F7D80,#E2BAB0,#FF96B7,#5FD3C8
517,Endless Cubes,#34DED0,#94FAF0,#3AF7F0,#31D1D0
518,Fog and Waterfall,#756457,#3B3C38,#D5BBA9,#F1ECE4


### Function to calculate hex to rgb

In [4]:
def convert_hex_to_rgb(input_hex):
  h = input_hex.lstrip('#')
  return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

In [5]:
def rgb_to_hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

# Saving out the PNG images

### Function to save out the color images of the color palette.

In [6]:
def save_color_images(df):

  # Ensure the folder exists
  if not os.path.exists(canva_img_folder):
      os.makedirs(canva_img_folder, exist_ok=True)

  # Iterate through DataFrame rows
  for index, row in df.iterrows():
    #Define Image size - height, width and depth
    height, width, channel = 300, 300, 3

    color_col = ['Color 1', 'Color 2', 'Color 3', 'Color 4']

    for color in color_col:

      color_name = row[color].replace("#","")

      red, green, blue = convert_hex_to_rgb(row[color])

      #Generate RGB Numpy Array 
      image_data = np.full((height, width, channel), [red, green, blue], dtype=('uint8'))

      plt.imshow(image_data)
      plt.axis('off')

      # Save the figure
      plt.savefig(f"{canva_img_folder}/{color_name}.png", bbox_inches='tight', pad_inches=0)
      plt.close()

In [7]:
start_time = time.time()

save_color_images(product_df)

end_time = time.time()

runtime = end_time - start_time
print("Generating Color PNG:", runtime, "seconds")

Generating Color PNG: 54.297821044921875 seconds


In [8]:
def save_to_hdf5(processed_images, filename):
    with h5py.File(filename, 'w') as f:
        for i, (img, flat) in enumerate(processed_images):
            f.create_dataset(f'image_{i}', data=img)
            f.create_dataset(f'flat_{i}', data=flat)


In [9]:
def load_from_hdf5(filename):
    with h5py.File(filename, 'r') as f:
        # Initialize containers for images and flattened data
        images = []
        flattened_data = []

        # Iterate over items in HDF5 file and load them
        for i in range(len(f.keys()) // 2):  # Assuming each image has two corresponding keys (image and flat)
            img_key = f'image_{i}'
            flat_key = f'flat_{i}'
            
            # Load and append to respective lists
            images.append(np.array(f[img_key]))  # Convert to numpy array if necessary
            flattened_data.append(np.array(f[flat_key]))

    return images, flattened_data


# Preprocess the images 

### Function to preprocess the input man

In [10]:
# Function to preprocess and flatten images
def preprocess(image_path):
    img = imread(image_path)
    if  img.shape[2] == 4:  # Check if the image has an alpha channel
        img = img[:, :, :3]  # Remove the alpha channel if present

    # Convert the RGB image to Lab color space
    #img_lab = rgb2hsv(img)
    img_lab = rgb2lab(img)

    # Resize the image to a fixed size (e.g., 256x256)
    img_resized = resize(img_lab, (256, 256), anti_aliasing=True)
    
    # Optionally, you can flatten the image if needed for further processing
    img_flattened = img_resized.flatten()

    # Return both the resized Lab image and the flattened version
    return img_resized, img_flattened

In [11]:
image_list= []

for image_file in os.listdir(canva_img_folder):
  image_list.append(os.path.join(canva_img_folder,image_file))

In [12]:
processed_images = [preprocess(img_path) for img_path in image_list]
processed_images

[(array([[[94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          ...,
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242]],
  
         [[94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          ...,
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242]],
  
         [[94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          ...,
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242],
          [94.10001261,  0.2013199 , -1.55898242]],
  
         ...,
  
        

# Saving preprocessed canva palattes to a h5 file

In [13]:
save_to_hdf5(processed_images, os.path.join(data_folder,'h5','preprocessed_canva_palettes.h5'))

---

### Next Notebook: [3.1 Matching Input Photo to Products](3.1_matching_input_photo_to_products.ipynb)